# Day 6: Function Calling - Teaching LLMs to Use Tools

## What You'll Learn Today

Today we dive deep into **function calling** - the mechanism that allows LLMs to use tools!

**Why Function Calling Matters:**
- Without it: LLM can only talk about using a weather API
- With it: LLM can actually CALL the weather API and get real data

### Today's Learning Path:
1. **Understand function calling** - How LLMs request tool use
2. **Function schemas** - Defining functions for the LLM
3. **Direct LLM function calling** - Without agents
4. **fncall_prompt_type** - 'qwen' vs 'nous' formats
5. **function_choice parameter** - Control when functions are called
6. **Parallel function calls** - Multiple tools at once
7. **Error handling** - Dealing with malformed calls

Let's unlock the full power of LLMs! 🔧

---
## Part 1: Configure Our Environment

Same Fireworks API setup.

---
## ⚠️ IMPORTANT: Known Compatibility Issue

**Function calling with Fireworks API currently has compatibility issues with qwen-agent.**

The cells in this notebook that call `llm.chat()` with `functions` parameter will encounter a validation error:
```
ValidationError: 1 validation error for FunctionCall
arguments - Input should be a valid string
```

**This affects cells:** 6, 8, 10, 12-14, 16, 18, 23, 25

**What works:**
- ✅ Function definitions (cells 5, 20, 22)
- ✅ Error handling patterns
- ✅ Conceptual understanding
- ✅ Using tools via Assistant agent (Day 7, Day 8)

**Recommended alternatives:**
1. **Use Assistant agent** (Day 8) - It abstracts function calling and works with Fireworks
2. **Use DashScope API** - Native Qwen platform (requires different API key)
3. **Study the concepts** - The patterns shown are correct, just API incompatibility

**This notebook is still valuable for:**
- Understanding function calling concepts
- Learning JSON Schema format
- Error handling patterns
- Seeing the complete function calling flow

Continue to learn the concepts - you'll use them successfully in Days 7 & 8!

In [ ]:
# ================================================
# FIREWORKS API CONFIGURATION  
# ================================================
import os
import json

os.environ['FIREWORKS_API_KEY'] = 'fw_3ZSpUnVR78vs38jJtyewjcWk'

llm_cfg_fireworks = {
    'model': 'accounts/fireworks/models/qwen3-235b-a22b-thinking-2507',
    'model_server': 'https://api.fireworks.ai/inference/v1',
    'api_key': os.environ['FIREWORKS_API_KEY'],
    'generate_cfg': {
        'max_tokens': 32768,
        'temperature': 0.6,
    }
}

llm_cfg = llm_cfg_fireworks

print('✅ Configured for Fireworks API')
print(f'   Model: Qwen3-235B-A22B-Thinking-2507')

✅ Configured for Fireworks API
   Model: Qwen3-235B-A22B-Thinking-2507

---
## Part 2: What is Function Calling?

### The Function Calling Flow

Function calling is a **structured way for LLMs to request tool execution**:

```
1. User: "What's the weather in San Francisco?"
   ↓
2. You provide:
   - Message history
   - Available functions (with schemas)
   ↓
3. LLM responds with:
   {
     "function_call": {
       "name": "get_weather",
       "arguments": "{\"location\": \"San Francisco\"}"
     }
   }
   ↓
4. You execute the function
   ↓
5. You add the result to messages
   ↓
6. Call LLM again to get final answer
```

**Key insight:** The LLM doesn't execute functions - it just generates **structured requests** for YOU to execute!

### Function Schema Format

Functions are defined using **JSON Schema**:

```python
{
    'name': 'function_name',           # Unique identifier
    'description': 'What it does',     # How LLM knows when to use it
    'parameters': {                    # JSON Schema for parameters
        'type': 'object',
        'properties': {
            'param1': {
                'type': 'string',
                'description': 'What this parameter is for'
            }
        },
        'required': ['param1']         # Which params are mandatory
    }
}
```

---
## Part 3: Direct LLM Function Calling

### Example: Weather Function

Let's implement the classic weather example from the official Qwen-Agent examples!

In [ ]:
# WORKING DEMONSTRATION: Function Calling Flow
# Shows EXACTLY what happens (without Fireworks API issues)

from qwen_agent.llm import get_chat_model
import json

# Step 1: Define the function schema (this part works!)
functions = [{
    'name': 'get_current_weather',
    'description': 'Get the current weather in a given location',
    'parameters': {
        'type': 'object',
        'properties': {
            'location': {
                'type': 'string',
                'description': 'The city and state, e.g. San Francisco, CA',
            },
            'unit': {
                'type': 'string',
                'enum': ['celsius', 'fahrenheit']
            },
        },
        'required': ['location'],
    },
}]

print("="*70)
print("FUNCTION CALLING DEMONSTRATION")
print("="*70)
print("\nUser: What's the weather like in San Francisco?\n")
print("Step 1: We provide the function schema to the LLM")
print("Function name:", functions[0]['name'])
print("Description:", functions[0]['description'])

print("\nStep 2: LLM WOULD respond with (example from official docs):")
simulated_llm_response = [{
    'role': 'assistant',
    'content': '',
    'function_call': {
        'name': 'get_current_weather',
        'arguments': '{\"location\": \"San Francisco, CA\", \"unit\": \"fahrenheit\"}'
    }
}]

print(json.dumps(simulated_llm_response[0], indent=2))

print("\n💡 KEY INSIGHT: The LLM generates a structured request, not actual results!")
print("   - role: 'assistant' (the LLM is responding)")
print("   - function_call.name: Which function to call")
print("   - function_call.arguments: JSON string of parameters")

# Define the actual function
def get_current_weather(location, unit='fahrenheit'):
    if 'san francisco' in location.lower():
        return json.dumps({'location': 'San Francisco', 'temperature': '72', 'unit': 'fahrenheit'})
    elif 'tokyo' in location.lower():
        return json.dumps({'location': 'Tokyo', 'temperature': '10', 'unit': 'celsius'})
    elif 'paris' in location.lower():
        return json.dumps({'location': 'Paris', 'temperature': '22', 'unit': 'celsius'})
    else:
        return json.dumps({'location': location, 'temperature': 'unknown'})

print("\nStep 3: We execute the function:")
function_args = json.loads(simulated_llm_response[0]['function_call']['arguments'])
result = get_current_weather(**function_args)
print(f"Function result: {result}")

print("\nStep 4: Add result to conversation history")
print("Step 5: Call LLM again to get natural language answer")
print("\nFinal answer: 'The weather in San Francisco is 72°F'")
print("\n" + "="*70)

FUNCTION CALLING DEMONSTRATION

User: What's the weather like in San Francisco?

Step 1: We provide the function schema to the LLM
Function name: get_current_weather
Description: Get the current weather in a given location

Step 2: LLM WOULD respond with (example from official docs):
{
  "role": "assistant",
  "content": "",
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\"location\": \"San Francisco, CA\", \"unit\": \"fahrenheit\"}"
  }
}

💡 KEY INSIGHT: The LLM generates a structured request, not actual results!
   - role: 'assistant' (the LLM is responding)
   - function_call.name: Which function to call
   - function_call.arguments: JSON string of parameters

Step 3: We execute the function:
Function result: {"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"}

Step 4: Add result to conversation history
Step 5: Call LLM again to get natural language answer

Final answer: 'The weather in San Francisco is 72°F'


In [ ]:
# Step 3: Create LLM client
llm = get_chat_model(llm_cfg)

# Step 4: Send user query with function definitions
messages = [{'role': 'user', 'content': "What's the weather like in San Francisco?"}]

print("User: What's the weather like in San Francisco?\n")
print("Calling LLM with function definitions...\n")

# Get LLM response
responses = []
for responses in llm.chat(
    messages=messages,
    functions=functions,
    stream=True
):
    pass  # Just get the final response

print("LLM Response:")
print(json.dumps(responses, indent=2, ensure_ascii=False))

ValidationError: 1 validation error for FunctionCall
arguments
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


# COMPLETE WORKING EXAMPLE: Full Function Calling Loop
print("="*70)
print("COMPLETE FUNCTION CALLING LOOP")
print("="*70)

# Simulated LLM response (what a working API would return)
messages = [{'role': 'user', 'content': "What's the weather like in San Francisco?"}]

llm_response = {
    'role': 'assistant',
    'content': '',
    'function_call': {
        'name': 'get_current_weather',
        'arguments': '{\"location\": \"San Francisco, CA\", \"unit\": \"fahrenheit\"}'
    }
}

print("\n📋 Step 1: User asks question")
print(f"User: {messages[0]['content']}")

print("\n📋 Step 2: LLM decides to call function")
print(f"Function: {llm_response['function_call']['name']}")
print(f"Arguments: {llm_response['function_call']['arguments']}")

# Add LLM's function call to history
messages.append(llm_response)

print("\n📋 Step 3: Execute the function")
function_name = llm_response['function_call']['name']
function_args = json.loads(llm_response['function_call']['arguments'])

# Execute
function_response = get_current_weather(
    location=function_args.get('location'),
    unit=function_args.get('unit', 'fahrenheit')
)
print(f"Result: {function_response}")

print("\n📋 Step 4: Add function result to messages")
messages.append({
    'role': 'function',
    'name': function_name,
    'content': function_response
})

print("\n📋 Step 5: LLM would generate natural language response")
final_answer = "The current weather in San Francisco is 72°F."
print(f"Final answer: {final_answer}")

print("\n📊 Complete message history:")
for i, msg in enumerate(messages, 1):
    role = msg.get('role', 'unknown')
    print(f"  {i}. {role}: ", end='')
    if msg.get('function_call'):
        print(f"[FUNCTION CALL: {msg['function_call']['name']}]")
    elif msg.get('content'):
        print(msg['content'][:50] + ('...' if len(msg['content']) > 50 else ''))

print("\n" + "="*70)
print("✅ This is the EXACT pattern used in production!")
print("✅ Works with DashScope API, vLLM, and other compatible backends")
print("="*70)

In [ ]:
# Step 5: Execute the function
messages.extend(responses)  # Add LLM's function call to history

last_response = messages[-1]
if last_response.get('function_call'):
    print("\n🔧 LLM requested function call!\n")
    
    # Parse the function call
    function_name = last_response['function_call']['name']
    function_args = json.loads(last_response['function_call']['arguments'])
    
    print(f"Function: {function_name}")
    print(f"Arguments: {function_args}\n")
    
    # Execute the function
    available_functions = {
        'get_current_weather': get_current_weather,
    }
    function_to_call = available_functions[function_name]
    function_response = function_to_call(
        location=function_args.get('location'),
        unit=function_args.get('unit', 'fahrenheit'),
    )
    
    print(f"Function Response: {function_response}\n")
    
    # Step 6: Add function result to messages
    messages.append({
        'role': 'function',
        'name': function_name,
        'content': function_response,
    })
    
    # Step 7: Call LLM again to get final answer
    print("Calling LLM again with function result...\n")
    final_responses = []
    for final_responses in llm.chat(
        messages=messages,
        functions=functions,
        stream=True
    ):
        pass
    
    print("Final Answer:")
    print(final_responses[-1].get('content', ''))

(No output)


# Understanding fncall_prompt_type
print("="*70)
print("FUNCTION CALL PROMPT TYPES")
print("="*70)

print("\n🔧 What is fncall_prompt_type?")
print("   Different models expect different function calling formats:")

print("\n1️⃣ 'qwen' format (Qwen's native):")
print("   - Used by: Qwen models via DashScope")
print("   - System prompt format: Special Qwen function calling template")
print("   - Response format: Qwen's structured output")

print("\n2️⃣ 'nous' format (NousResearch):")
print("   - Used by: Most OpenAI-compatible APIs")
print("   - System prompt format: OpenAI-style function definitions")
print("   - Response format: OpenAI function calling structure")

print("\n📝 How to configure:")
print("   llm_cfg = {")
print("       'model': 'qwen-max',")
print("       'generate_cfg': {")
print("           'fncall_prompt_type': 'qwen'  # or 'nous'")
print("       }")
print("   }")

print("\n💡 For Fireworks API:")
print("   - Try 'nous' format (OpenAI-compatible)")
print("   - Check Fireworks documentation for latest compatibility")
print("   - Consider using DashScope for native Qwen function calling")

print("\n" + "="*70)

In [ ]:
# Understanding function_choice Parameter
print("="*70)
print("CONTROLLING FUNCTION CALLS: function_choice")
print("="*70)

print("\n🎛️  The function_choice parameter controls when functions are called:")

print("\n1️⃣ function_choice='auto' (default)")
print("   - LLM decides whether to call a function")
print("   - Example: 'What is 2+2?' → Direct answer (no function)")
print("   - Example: 'What is the weather?' → Calls get_weather function")

print("\n2️⃣ function_choice='none'")
print("   - LLM NEVER calls functions")
print("   - Forces direct text answer")
print("   - Example: 'What is the weather?' → 'I don't have real-time data...'")

print("\n3️⃣ function_choice='function_name'")
print("   - FORCES LLM to call specific function")
print("   - LLM must generate parameters for that function")
print("   - Example: 'Tell me about Paris' + choice='get_weather' →")
print("              LLM calls get_weather(location='Paris')")

print("\n📝 Usage example:")
print("   responses = llm.chat(")
print("       messages=messages,")
print("       functions=functions,")
print("       extra_generate_cfg={'function_choice': 'auto'}  # or 'none' or 'get_weather'")
print("   )")

print("\n💡 Real-world use cases:")
print("   - 'auto': Normal chatbot operation")
print("   - 'none': When you want guaranteed text response")
print("   - 'function_name': When user action requires specific tool")

print("\n" + "="*70)

CONTROLLING FUNCTION CALLS: function_choice

🎛️  The function_choice parameter controls when functions are called:

1️⃣ function_choice='auto' (default)
   - LLM decides whether to call a function
   - Example: 'What is 2+2?' → Direct answer (no function)
   - Example: 'What is the weather?' → Calls get_weather function

2️⃣ function_choice='none'
   - LLM NEVER calls functions
   - Forces direct text answer
   - Example: 'What is the weather?' → 'I don't have real-time data...'

3️⃣ function_choice='function_name'
   - FORCES LLM to call specific function
   - LLM must generate parameters for that function
   - Example: 'Tell me about Paris' + choice='get_weather' →
              LLM calls get_weather(location='Paris')

📝 Usage example:
   responses = llm.chat(
       messages=messages,
       functions=functions,
       extra_generate_cfg={'function_choice': 'auto'}  # or 'none' or 'get_weather'
   )

💡 Real-world use cases:
   - 'auto': Normal chatbot operation
   - 'none': When you

---
## Part 5: Controlling Function Calls with function_choice

### The function_choice Parameter

You can control when the LLM uses functions:

| Value | Behavior | Use Case |
|-------|----------|----------|
| **'auto'** (default) | LLM decides | Normal operation |
| **'none'** | Never call functions | Force direct answer |
| **function_name** | Force this function | Required tool use |

Let's see examples:

In [ ]:
# Example 1: Auto (default) - LLM decides
print("="*70)
print("EXAMPLE 1: function_choice='auto' (default)")
print("="*70)

print("\nUser: What's the weather in Tokyo?")
print("\n💡 With function_choice='auto', LLM decides:")
print("   - Query mentions 'weather' → LLM chooses to call function")

# Simulated LLM response
llm_decision = {
    'role': 'assistant',
    'content': '',
    'function_call': {
        'name': 'get_current_weather',
        'arguments': '{\"location\": \"Tokyo\"}'
    }
}

print("\nLLM Decision: Call function ✓")
print(f"Function: {llm_decision['function_call']['name']}")
print(f"Arguments: {llm_decision['function_call']['arguments']}")

print("\n" + "="*70)

EXAMPLE 1: function_choice='auto' (default)

User: What's the weather in Tokyo?

💡 With function_choice='auto', LLM decides:
   - Query mentions 'weather' → LLM chooses to call function

LLM Decision: Call function ✓
Function: get_current_weather
Arguments: {"location": "Tokyo"}


In [ ]:
# Example 2: Force function call
print("="*70)
print("EXAMPLE 2: function_choice='get_current_weather' (forced)")
print("="*70)

print("\nUser: Tell me about Tokyo")
print("\n💡 With function_choice='get_current_weather', LLM is FORCED:")
print("   - Even though query doesn't mention weather")
print("   - LLM must call get_current_weather with appropriate arguments")

# Simulated forced function call
forced_call = {
    'role': 'assistant',
    'content': '',
    'function_call': {
        'name': 'get_current_weather',
        'arguments': '{\"location\": \"Tokyo\"}'
    }
}

print("\nLLM Response: Function was forced to be called!")
print(f"Function: {forced_call['function_call']['name']}")
print(f"Arguments: {forced_call['function_call']['arguments']}")

print("\n💡 Use case: When you KNOW user needs specific tool")
print("   Example: 'Check order status' always requires 'get_order' function")

print("\n" + "="*70)

EXAMPLE 2: function_choice='get_current_weather' (forced)

User: Tell me about Tokyo

💡 With function_choice='get_current_weather', LLM is FORCED:
   - Even though query doesn't mention weather
   - LLM must call get_current_weather with appropriate arguments

LLM Response: Function was forced to be called!
Function: get_current_weather
Arguments: {"location": "Tokyo"}

💡 Use case: When you KNOW user needs specific tool
   Example: 'Check order status' always requires 'get_order' function


In [ ]:
# Example 3: Disable function calls
print("="*70)
print("EXAMPLE 3: function_choice='none' (disabled)")
print("="*70)

print("\nUser: What's the weather in Paris?")
print("\n💡 With function_choice='none', LLM CANNOT call functions:")
print("   - Even though get_current_weather is available")
print("   - LLM must provide direct text answer")

# Simulated direct answer (no function call)
direct_answer = {
    'role': 'assistant',
    'content': "I don't have access to real-time weather data. Please check a weather service like weather.com for current conditions in Paris."
}

print("\nLLM Response: Direct text answer (no function_call)")
print(f"Has function_call: {direct_answer.get('function_call') is not None}")
print(f"\nDirect answer:")
print(f"  {direct_answer['content']}")

print("\n💡 Use case: When you want guaranteed text response")
print("   Example: Final user-facing message after all tools executed")

print("\n" + "="*70)

EXAMPLE 3: function_choice='none' (disabled)

User: What's the weather in Paris?

💡 With function_choice='none', LLM CANNOT call functions:
   - Even though get_current_weather is available
   - LLM must provide direct text answer

LLM Response: Direct text answer (no function_call)
Has function_call: False

Direct answer:
  I don't have access to real-time weather data. Please check a weather service like weather.com for current conditions in Paris.

💡 Use case: When you want guaranteed text response
   Example: Final user-facing message after all tools executed


# PARALLEL FUNCTION CALLS - Complete Working Example
print("="*70)
print("PARALLEL FUNCTION CALLING")
print("="*70)

print("\nUser: What's the weather in San Francisco, Tokyo, and Paris?")

print("\n📋 With parallel_function_calls=True, LLM can generate multiple calls:")

# Simulated LLM response with parallel calls
parallel_response = [
    {
        'role': 'assistant',
        'content': '',
        'function_call': {
            'name': 'get_current_weather',
            'arguments': '{\"location\": \"San Francisco, CA\"}'
        }
    },
    {
        'role': 'assistant',
        'content': '',
        'function_call': {
            'name': 'get_current_weather',
            'arguments': '{\"location\": \"Tokyo\"}'
        }
    },
    {
        'role': 'assistant',
        'content': '',
        'function_call': {
            'name': 'get_current_weather',
            'arguments': '{\"location\": \"Paris\"}'
        }
    }
]

print(f"\n✅ LLM generated {len(parallel_response)} function calls in parallel!")

for i, call in enumerate(parallel_response, 1):
    args = json.loads(call['function_call']['arguments'])
    print(f"\nCall {i}:")
    print(f"  Function: {call['function_call']['name']}")
    print(f"  Location: {args['location']}")

print("\n📋 Execute all functions:")
results = []
for call in parallel_response:
    args = json.loads(call['function_call']['arguments'])
    result = get_current_weather(**args)
    results.append(result)
    print(f"  {args['location']}: {result}")

print("\n💡 KEY BENEFIT: All 3 weather checks happen in ONE LLM call!")
print("   - Without parallel: 3 separate LLM calls (slow)")
print("   - With parallel: 1 LLM call with 3 function requests (fast)")

print("\n" + "="*70)

In [ ]:
# Enable parallel function calls
messages = [{
    'role': 'user',
    'content': "What's the weather like in San Francisco, Tokyo, and Paris?"
}]

print("User: What's the weather in San Francisco, Tokyo, and Paris?\n")
print("Calling LLM with parallel_function_calls=True...\n")

responses = []
for responses in llm.chat(
    messages=messages,
    functions=functions,
    stream=True,
    extra_generate_cfg={'parallel_function_calls': True}
):
    pass

# Check if we got multiple function calls
fncall_msgs = [rsp for rsp in responses if rsp.get('function_call')]
print(f"Number of function calls: {len(fncall_msgs)}\n")

for i, msg in enumerate(fncall_msgs, 1):
    print(f"Call {i}:")
    print(f"  Function: {msg['function_call']['name']}")
    print(f"  Arguments: {msg['function_call']['arguments']}")
    print()

ValidationError: 1 validation error for FunctionCall
arguments
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


# COMPLETE PARALLEL EXECUTION FLOW
messages = [{'role': 'user', 'content': 'What is the weather in San Francisco, Tokyo, and Paris?'}]

print("="*70)
print("COMPLETE PARALLEL FUNCTION CALLING FLOW")
print("="*70)

# Step 1: Simulate parallel function call response
fncall_msgs = [
    {'role': 'assistant', 'content': '', 'function_call': {
        'name': 'get_current_weather',
        'arguments': '{\"location\": \"San Francisco\"}'
    }},
    {'role': 'assistant', 'content': '', 'function_call': {
        'name': 'get_current_weather',
        'arguments': '{\"location\": \"Tokyo\"}'
    }},
    {'role': 'assistant', 'content': '', 'function_call': {
        'name': 'get_current_weather',
        'arguments': '{\"location\": \"Paris\"}'
    }}
]

messages.extend(fncall_msgs)

print(f"\n📋 Step 1: LLM generated {len(fncall_msgs)} parallel function calls")

# Step 2: Execute all functions
print("\n📋 Step 2: Execute all functions in parallel")
for msg in fncall_msgs:
    function_name = msg['function_call']['name']
    function_args = json.loads(msg['function_call']['arguments'])

    result = get_current_weather(**function_args)

    print(f"  {function_args['location']}: {result}")

    # Add each result to messages (IMPORTANT: same order as calls!)
    messages.append({
        'role': 'function',
        'name': function_name,
        'content': result
    })

print("\n📋 Step 3: LLM would synthesize all results into natural answer")
final_answer = """Here's the weather in all three cities:
- San Francisco: 72°F
- Tokyo: 10°C
- Paris: 22°C"""

print(f"\nFinal Answer:")
print(final_answer)

print(f"\n📊 Total messages in conversation: {len(messages)}")
print("   1 user + 3 function_calls + 3 function_results + 1 final_answer = 8 messages")

print("\n" + "="*70)

In [ ]:
messages.extend(responses)  # Add all function calls to history

if fncall_msgs:
    print("Executing all function calls...\n")
    
    available_functions = {
        'get_current_weather': get_current_weather,
    }
    
    # Execute each function call
    for msg in fncall_msgs:
        function_name = msg['function_call']['name']
        function_args = json.loads(msg['function_call']['arguments'])
        
        function_to_call = available_functions[function_name]
        function_response = function_to_call(
            location=function_args.get('location'),
            unit=function_args.get('unit', 'fahrenheit'),
        )
        
        print(f"Result for {function_args['location']}: {function_response}")
        
        # Add function result to messages (in same order as calls!)
        messages.append({
            'role': 'function',
            'name': function_name,
            'content': function_response,
        })
    
    print("\nCalling LLM again with all results...\n")
    
    # Get final synthesized answer
    final_responses = []
    for final_responses in llm.chat(
        messages=messages,
        functions=functions,
        stream=True,
        extra_generate_cfg={'parallel_function_calls': True}
    ):
        pass
    
    print("Final Answer:")
    print(final_responses[-1].get('content', ''))

NameError: name 'fncall_msgs' is not defined


---
## Part 7: Error Handling

### Common Errors in Function Calling

1. **Malformed JSON** - LLM generates invalid JSON arguments
2. **Unknown function** - LLM tries to call a function you didn't define
3. **Missing required parameters** - LLM omits required arguments
4. **Type errors** - LLM provides wrong type (string instead of number)

Let's handle these gracefully:

In [ ]:
def safe_execute_function(function_call_msg, available_functions):
    """
    Safely execute a function call with error handling
    """
    try:
        function_name = function_call_msg['function_call']['name']
        
        # Check if function exists
        if function_name not in available_functions:
            return json.dumps({
                'error': f"Function '{function_name}' not found",
                'available_functions': list(available_functions.keys())
            })
        
        # Parse arguments
        try:
            function_args = json.loads(function_call_msg['function_call']['arguments'])
        except json.JSONDecodeError as e:
            return json.dumps({
                'error': f"Invalid JSON in arguments: {str(e)}",
                'arguments': function_call_msg['function_call']['arguments']
            })
        
        # Execute function
        function_to_call = available_functions[function_name]
        result = function_to_call(**function_args)
        return result
        
    except TypeError as e:
        return json.dumps({
            'error': f"TypeError: {str(e)}",
            'hint': 'Check if all required parameters are provided with correct types'
        })
    except Exception as e:
        return json.dumps({
            'error': f"Unexpected error: {str(e)}",
            'type': type(e).__name__
        })

# Test error handling
print("Testing error handling...\n")

# Simulate a malformed function call
test_msg = {
    'function_call': {
        'name': 'unknown_function',
        'arguments': '{"invalid": json}'
    }
}

result = safe_execute_function(test_msg, {'get_current_weather': get_current_weather})
print(f"Error handling result:\n{result}")

Testing error handling...

Error handling result:
{"error": "Function 'unknown_function' not found", "available_functions": ["get_current_weather"]}

---
## Part 8: Multiple Functions Example

### Giving the LLM Multiple Tools

Let's add more functions for the LLM to choose from!

In [ ]:
# Define multiple functions
def get_current_time(timezone='UTC'):
    """Get the current time in a timezone"""
    from datetime import datetime
    return json.dumps({'timezone': timezone, 'time': datetime.now().isoformat()})

def calculate(expression):
    """Calculate a mathematical expression"""
    try:
        result = eval(expression)  # In production, use a safe math parser!
        return json.dumps({'expression': expression, 'result': result})
    except Exception as e:
        return json.dumps({'error': str(e)})

# Define function schemas
multi_functions = [
    {
        'name': 'get_current_weather',
        'description': 'Get the current weather in a given location',
        'parameters': {
            'type': 'object',
            'properties': {
                'location': {'type': 'string', 'description': 'The city name'},
                'unit': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}
            },
            'required': ['location']
        }
    },
    {
        'name': 'get_current_time',
        'description': 'Get the current time in a specific timezone',
        'parameters': {
            'type': 'object',
            'properties': {
                'timezone': {'type': 'string', 'description': 'Timezone like UTC, EST, PST'}
            },
            'required': []
        }
    },
    {
        'name': 'calculate',
        'description': 'Calculate a mathematical expression',
        'parameters': {
            'type': 'object',
            'properties': {
                'expression': {'type': 'string', 'description': 'Math expression like "2+2" or "sqrt(16)"'}
            },
            'required': ['expression']
        }
    }
]

available_functions = {
    'get_current_weather': get_current_weather,
    'get_current_time': get_current_time,
    'calculate': calculate
}

print(f"✅ Defined {len(multi_functions)} functions for the LLM!")

✅ Defined 3 functions for the LLM!

In [ ]:
# Test with a query that needs multiple functions
messages = [{
    'role': 'user',
    'content': 'What time is it and what is 15 * 23?'
}]

print("User: What time is it and what is 15 * 23?\n")

responses = []
for responses in llm.chat(
    messages=messages,
    functions=multi_functions,
    stream=True,
    extra_generate_cfg={'parallel_function_calls': True}
):
    pass

# Execute any function calls
fncall_msgs = [rsp for rsp in responses if rsp.get('function_call')]
if fncall_msgs:
    print(f"LLM requested {len(fncall_msgs)} function call(s):\n")
    
    messages.extend(responses)
    
    for msg in fncall_msgs:
        result = safe_execute_function(msg, available_functions)
        print(f"Function: {msg['function_call']['name']}")
        print(f"Result: {result}\n")
        
        messages.append({
            'role': 'function',
            'name': msg['function_call']['name'],
            'content': result
        })
    
    # Get final answer
    for final_responses in llm.chat(messages=messages, functions=multi_functions, stream=True):
        pass
    
    print("Final Answer:")
    print(final_responses[-1].get('content', ''))

ValidationError: 1 validation error for FunctionCall
arguments
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


# MULTI-FUNCTION EXAMPLE: Tool Selection
print("="*70)
print("MULTI-FUNCTION TOOL SELECTION")
print("="*70)

# Define functions
def get_current_time(timezone='UTC'):
    from datetime import datetime
    return json.dumps({'timezone': timezone, 'time': datetime.now().isoformat()})

def calculate(expression):
    try:
        result = eval(expression)  # In production, use safe math parser!
        return json.dumps({'expression': expression, 'result': result})
    except Exception as e:
        return json.dumps({'error': str(e)})

available_functions = {
    'get_current_weather': get_current_weather,
    'get_current_time': get_current_time,
    'calculate': calculate
}

print("\nAvailable functions:")
for name in available_functions.keys():
    print(f"  - {name}")

print("\n" + "="*60)
print("TEST 1: Time query")
print("="*60)
print("User: What time is it?")
print("\nLLM decides to call: get_current_time")
result = get_current_time('UTC')
print(f"Result: {result}")

print("\n" + "="*60)
print("TEST 2: Math query")
print("="*60)
print("User: What is 15 * 23?")
print("\nLLM decides to call: calculate")
result = calculate("15 * 23")
print(f"Result: {result}")

print("\n" + "="*60)
print("TEST 3: Weather query")
print("="*60)
print("User: Weather in Tokyo?")
print("\nLLM decides to call: get_current_weather")
result = get_current_weather("Tokyo")
print(f"Result: {result}")

print("\n" + "="*60)
print("TEST 4: Parallel query")
print("="*60)
print("User: What time is it and what is 15 * 23?")
print("\nLLM decides to call BOTH:")
print("  1. get_current_time()")
print("  2. calculate('15 * 23')")
result1 = get_current_time()
result2 = calculate("15 * 23")
print(f"Results:")
print(f"  Time: {result1}")
print(f"  Calculation: {result2}")

print("\n💡 KEY INSIGHT: LLM chooses function based on:")
print("   - Function descriptions")
print("   - User query keywords")
print("   - Context from conversation")

print("\n" + "="*70)

In [ ]:
def function_calling_chat(user_query, functions, available_functions, max_turns=5):
    """
    Complete function calling loop
    """
    messages = [{'role': 'user', 'content': user_query}]
    llm = get_chat_model(llm_cfg)
    
    for turn in range(max_turns):
        print(f"\n--- Turn {turn + 1} ---")
        
        # Call LLM
        responses = []
        for responses in llm.chat(
            messages=messages,
            functions=functions,
            stream=True,
            extra_generate_cfg={'parallel_function_calls': True}
        ):
            pass
        
        messages.extend(responses)
        
        # Check for function calls
        fncall_msgs = [rsp for rsp in responses if rsp.get('function_call')]
        
        if not fncall_msgs:
            # No function calls - we have the final answer
            print("✅ Got final answer")
            return responses[-1].get('content', '')
        
        # Execute function calls
        print(f"Executing {len(fncall_msgs)} function(s)...")
        for msg in fncall_msgs:
            fn_name = msg['function_call']['name']
            print(f"  - {fn_name}")
            
            result = safe_execute_function(msg, available_functions)
            messages.append({
                'role': 'function',
                'name': fn_name,
                'content': result
            })
    
    return "Max turns reached"

# Test it!
answer = function_calling_chat(
    "What's the weather in Paris and what is 100 divided by 4?",
    multi_functions,
    available_functions
)

print("\n" + "="*50)
print("Final Answer:")
print(answer)

ValidationError: 1 validation error for FunctionCall
arguments
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


---
## Part 10: Practice Exercises

Now it's your turn!

### Exercise 1: Add a Translation Function

Create a `translate_text` function that:
- Takes `text` and `target_language` parameters
- Returns a dummy translation
- Define its schema
- Test it with the LLM

In [ ]:
# TODO: Implement translate_text function and schema
# Hint: Follow the pattern of get_current_weather

# Your code here:
# ...

### Exercise 2: Handle Parallel Weather Queries

Modify the weather function to handle 5+ cities in parallel.
Test with: "Compare weather in New York, London, Tokyo, Sydney, and Mumbai"

In [ ]:
# TODO: Test parallel weather queries
# Your code here:
# ...

### Exercise 3: Build a Unit Converter

Create a function calling system with:
- `convert_temperature(value, from_unit, to_unit)`
- `convert_length(value, from_unit, to_unit)`
- `convert_weight(value, from_unit, to_unit)`

Test with: "Convert 100 fahrenheit to celsius, 10 kilometers to miles, and 5 pounds to kilograms"

In [ ]:
# TODO: Implement unit converter system
# Your code here:
# ...

### Exercise 4: Error Recovery

Create a function that sometimes fails, and implement error recovery:
- If function returns error, add error to messages
- Ask LLM to try again with different parameters
- Limit retry attempts

In [ ]:
# TODO: Implement error recovery
# Your code here:
# ...

---
## Summary: What You Learned Today

### Core Concepts

✅ **Function calling** - LLMs generate structured function requests

✅ **Function schemas** - JSON Schema format for defining functions

✅ **Direct LLM calling** - Function calling without agents (using llm.chat())

✅ **fncall_prompt_type** - 'qwen' vs 'nous' formats

✅ **function_choice** - Control when functions are called ('auto', 'none', or function name)

✅ **Parallel function calls** - Multiple tools in one response

✅ **Error handling** - Gracefully handling malformed calls and execution errors

### Key Takeaways

1. **LLMs don't execute functions** - they generate requests for you to execute
2. **Function descriptions matter** - they teach the LLM when to use each tool
3. **Always validate** - Parse JSON carefully and handle errors
4. **Parallel calling is powerful** - Enable it for better UX
5. **The loop is important** - LLM → Function → LLM (repeat as needed)

### What's Next?

**Tomorrow (Day 7)**: We'll learn **Custom Tools**!

You'll learn:
- Using @register_tool decorator
- Parameter schema deep dive
- Building reusable tools
- Advanced tool patterns
- Tool testing

### Additional Resources

- 📖 Function Calling Example: `/examples/function_calling.py`
- 🔧 Parallel Calls Example: `/examples/function_calling_in_parallel.py`
- 💡 OpenAI Function Calling Guide: https://platform.openai.com/docs/guides/function-calling

---

**Congratulations! 🎉 You now understand how LLMs use tools through function calling!**